# Investigating and Cleaning the data

## Loading the data

All analytics projects start with data. We need to read in the data in some format that will make it easy to manipulate in Python. In this tutorial, we'll be using the **Pandas** library to work with our data. 

The DataFrame is one of Pandas' most important data structures. It's basically a way to store tabular data, where you can label the rows and the columns. Think of it as a spreadsheet you can control programmatically with Python.

Let's start by loading the data!

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from scipy.stats import mstats

In [2]:
# Add data/ in front of the csv file name
df = pd.read_csv('../data/hmda_lar.csv')

# We'll check if it worked by calling the .head() function to see the first 5 rows.
df.head()

,action_taken,action_taken_name,agency_code,agency_abbr,agency_name,applicant_ethnicity,applicant_ethnicity_name,applicant_income_000s,applicant_race_1,applicant_race_2,...,state_abbr,state_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,rate_spread,tract_to_msamd_income
0,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,71.0,5,NaN,...,NY,New York,68100,128,1794,1669.0,7.150000,4501,NaN,115.269997
1,3,Application denied by financial institution,7,HUD,Department of Housing and Urban Development,3,"Information not provided by applicant in mail,...",27.0,6,NaN,...,NY,New York,72600,210,841,478.0,94.440002,2160,NaN,94.160004
2,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,98.0,5,NaN,...,NY,New York,64800,224,2375,893.0,3.190000,3355,1.71,87.650002
3,1,Loan originated,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,173.0,5,NaN,...,NY,New York,106200,254,765,738.0,21.730000,2453,NaN,100.330002
4,1,Loan originated,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,2640.0,5,NaN,...,NY,New York,72600,5180,327,1165.0,17.719999,5429,NaN,331.500000


There are some cool things going on here -- some very obvious and some hidden under the hood. Let's take it question by question.

<ul>
    <li><b>What is read_csv and how does it work? </b> read_csv is a predefined function that's part of the pandas package. It is able to take in a csv file, iterate through every row and column, and create a dataframe where you can apply powerful data analysis and visualization functions easily.</li>
    <li><b>What does "pd" stand for? </b> When we wrote <b>import</b> pandas <b>as</b> pd, we were importing the entire pandas library (i.e. all of the modules contained within the library) and making it accessible under the variable <i>pandas</i>. <i>pd</i> is simply an alias for pandas to limit the typing required. We could replace every pd with pandas and the code would work just fine. </li>
    <li><b>Why is there a "." before the read_csv function? </b> The "." is known as the dot notation and it applies many languages outside of Python. To fully explain the theory, we'd have to get into Object Oriented Programming, a paradigm in which everything is organized around objects rather that "actions." The pandas library, which we have just now called pd has many methods associated with it, one of which is the read_csv method. We access the methods associated with pd by using the dot ".". So the pandas read_csv method is just written pd.read_csv. This is an important concept to understand as you go on but for now, think of the dot notation as a way of specifying that you want to call the read_csv function from the pandas library.</li>
</ul>

We've used a csv file for the purposes of this example but it's important to note that there are a great number of other ways to receive data. Depending on what the clients have, we have to make sure we understand the format and requirements in order to be able to use it within the Python environment.

Notice that we have too many columns here to see the full range in this sample (there is a ... between applicant_race_2 and state_abbr indicating hidden columns). .info() can give you a view of all columns.

### APIs

We can sometimes use APIs to get data. Application Program Interfaces (API) are commonly used to retrieve data from remote websites. Sites like Reddit, Twitter, and Facebook all offer certain data through their APIs. To use an API, you make a request to a remote web server, and retrieve the data you need. 

We'll be using the API request below to get Below, we make an API request from the HMDA website to get the data we need. For the sake of speed, we'll only be looking at 100,000 data points from the 2016 New York data for the remaining modules.

In [3]:
state = "NY"
year = 2016
limit = 100000

# We get the first 100000 rows of loan application data from New York with as of year 2016. You can adjust the parameters!

base_url = ("https://api.consumerfinance.gov:443/data/hmda/slice/hmda_lar.json?"
            "$where=state_abbr='{state}'+AND+as_of_year={year}&$limit={limit}&$offset=0"
            .format(
                state =state,
                year = year,
                limit = limit
))

response = requests.get(base_url)
data = json.loads(response.content.decode('utf-8'))
raw_hmda_data = pd.DataFrame(data['results'])
raw_hmda_data

,action_taken,action_taken_name,agency_abbr,agency_code,agency_name,applicant_ethnicity,applicant_ethnicity_name,applicant_income_000s,applicant_race_1,applicant_race_2,...,property_type_name,purchaser_type,purchaser_type_name,rate_spread,respondent_id,sequence_number,state_abbr,state_code,state_name,tract_to_msamd_income
0,1,Loan originated,HUD,7,Department of Housing and Urban Development,2,Not Hispanic or Latino,71.0,5,NaN,...,One-to-four family dwelling (other than manufa...,6,"Commercial bank, savings bank or savings assoc...",,16-1566654,0001164,NY,36,New York,115.269997
1,3,Application denied by financial institution,HUD,7,Department of Housing and Urban Development,3,"Information not provided by applicant in mail,...",27.0,6,NaN,...,One-to-four family dwelling (other than manufa...,0,Loan was not originated or was not sold in cal...,,26-4599244,0277064,NY,36,New York,94.160004
2,1,Loan originated,HUD,7,Department of Housing and Urban Development,2,Not Hispanic or Latino,98.0,5,NaN,...,One-to-four family dwelling (other than manufa...,2,Ginnie Mae (GNMA),01.71,95-4623407,0025100,NY,36,New York,87.650002
3,1,Loan originated,CFPB,9,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,173.0,5,NaN,...,One-to-four family dwelling (other than manufa...,1,Fannie Mae (FNMA),,0000497404,0017124,NY,36,New York,100.330002
4,1,Loan originated,CFPB,9,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,2640.0,5,NaN,...,One-to-four family dwelling (other than manufa...,0,Loan was not originated or was not sold in cal...,,0000852218,0371343,NY,36,New York,331.500000
5,1,Loan originated,HUD,7,Department of Housing and Urban Development,2,Not Hispanic or Latino,56.0,5,NaN,...,One-to-four family dwelling (other than manufa...,6,"Commercial bank, savings bank or savings assoc...",,16-1566654,0000143,NY,36,New York,94.419998
6,6,Loan purchased by the institution,HUD,7,Department of Housing and Urban Development,4,Not applicable,NaN,7,NaN,...,One-to-four family dwelling (other than manufa...,3,Freddie Mac (FHLMC),,13-6131491,0185761,NY,36,New York,78.489998
7,1,Loan originated,HUD,7,Department of Housing and Urban Development,2,Not Hispanic or Latino,43.0,5,NaN,...,One-to-four family dwelling (other than manufa...,2,Ginnie Mae (GNMA),02.04,95-4623407,0000700,NY,36,New York,83.639999
8,1,Loan originated,FRS,2,Federal Reserve System,2,Not Hispanic or Latino,96.0,5,NaN,...,One-to-four family dwelling (other than manufa...,3,Freddie Mac (FHLMC),,0000592448,0001931,NY,36,New York,101.599998
9,6,Loan purchased by the institution,CFPB,9,Consumer Financial Protection Bureau,3,"Information not provided by applicant in mail,...",98.0,6,NaN,...,One-to-four family dwelling (other than manufa...,1,Fannie Mae (FNMA),,0000852218,0138303,NY,36,New York,149.309998


## Inspecting the data

In [ ]:
# Let's find out more about the dataframe itself.
raw_hmda_data.info()

In [ ]:
# You can also transpose the original dataset to see more of the columns
raw_hmda_data.head().T

In [ ]:
# You can also call .shape as another way to get the dimensions of the table you're working with.
raw_hmda_data.shape

In [ ]:
# Finally, you can call .describe to calculate basic statistics. Note that this only works on numeric columns
raw_hmda_data.describe()

A data dictionary was provided in section 3.1. These are essential to getting a good understanding of your data and the relationships within.

## Filtering and Selecting

We'll need to work with dataframes in order to extract information, create visualizations, and ultimately run our analysis. Below are a few exercises to play around with the dataset we've just loaded to help you get familiar.

#### Mini-exercise: Data transformation and manipulation<img style="float: left; width: 15px;" src=images/Design_Logo.png> 

Let's begin by working with the different ways to select data. Note that since we're only selecting, the raw_hmda_df will not actually be changed in any way. Therefore, we don't need to be wary of messing with the original data just yet.

In [ ]:
# You can select a single column from your data easily by referring to the column name in square brackets
# For example, we can select and show the action_taken_name column with: raw_hmda_data['action_taken_name'])
# Try showing the loan_amount_000s column only
## CODE HERE ##


When we output the single column,  each column is actually stored as a series, which is why it displays differently when you show it. This is why you need the extra set of square brackets in the next line to return a dataframe.

In [ ]:
# You can select multiple columns but another set of brackets is required to keep within the dataframe data type.
raw_hmda_data[['loan_amount_000s','hud_median_family_income']]

This is a very simple way to select entire columns. However, if you want to filter your dataFrame in smarter ways, we have a more powerful set of tools called iloc and loc.

**iloc**

Let's start with iloc. The iloc indexer for Pandas Dataframe is used for integer-location based indexing / selection by position. The syntax is `data.iloc[<row selection>, <column selection>]`. “iloc” in pandas is used to select rows and columns by number, in the order that they appear in the data frame. This is very similar to excel, where you index by row and column position numbers. You can imagine that each row and column has a number from 0 to the total row/total column and and iloc[] allows selections based on these numbers.

There are two “arguments” to iloc – a row selector, and a column selector.

In [ ]:
# Let's try selecting the first row of the dataset using iloc. If no column is specified, Python automatically returns all 
# columns associated with the specific row.
raw_hmda_data.iloc[0]

In [ ]:
# We can add in a column location as well to be more specific. In this example, we will select action_taken_name from the first
# row of the data. We should get 'Loan Originated' as the value. Note that we start counting from zero so action_taken_name 
# actually has a column number of 1.
## CODE HERE ##


In [ ]:
# We can also specify ranges to select within the brackets using ':'. The format is x:y, which means select from 
# element x to y. x: means select from x to end while :y means select from beginning to y. Just ':' means select all
# Here we select all columns of the first row to the 100th row.
## CODE HERE ##


When selecting multiple columns or multiple rows in this manner, remember that in your selection e.g.[1:5], the rows/columns selected will run from the first number to one minus the second number. e.g. [1:5] will go 1,2,3,4., [x,y] goes from x to y-1.

**loc**

Selections using the loc method are based on the index of the data frame (if any). Where the index is set on a DataFrame, the .loc method directly selects based on index values of any rows. The index can be changed using the set_index method but the default is generated for you when you create a new dataframe. In this case, the index is the column with no header at the very front, starting from 0.

In [ ]:
raw_hmda_data.head()

Because our current index starts from 0, it has the same label as the row position number. However, we would call on the column by name instead of position number.

In [ ]:
# Let's select the 'applicant_income_000s' value of the 2nd row. The index value would be 1. Note the column name must be a 
# string
raw_hmda_data.loc[1, 'applicant_income_000s']

However, if the rows were sorted in some order, the index value for each row would not change while the row position does. Therefore, .iloc and .loc would return different values. Let's use the first 5 rows of raw_hmda_data as an example.

In [ ]:
sample = raw_hmda_data.head()

# Sort in descending order by loan_amount_000s
ordered_sample = sample.sort_values('loan_amount_000s',ascending=False)
ordered_sample

See how the index follows the row? So now, .iloc and .loc using the same value for row will return different values.

In [ ]:
# Return the value at the 2nd row, 8th column position
ordered_sample.iloc[1,7]

In [ ]:
# Return the value at index 1, 'applicant_income_000s' column
ordered_sample.loc[1, 'applicant_income_000s']

Keeping this in mind, loc also allows for advanced selection methods.

In [ ]:
# Here we select all rows from 'action_taken' to 'agency_name' 
raw_hmda_data.loc[:,'action_taken':'agency_name']

In [ ]:
# Try printing all rows from the state_name column to the last column
## CODE HERE ##


In [ ]:
# You can also slice data based on certain conditions. For example, if you want to select all rows where agency_abbr = 'OCC', you
# can try the following
raw_hmda_data.loc[raw_hmda_data['agency_abbr']=='OCC']

In [ ]:
# Try selecting the subset of loan data using loc where agency_abbr = 'OCC' and as_of_year = 2016. Note that adding in 2 more 
# conditions has a specific syntax you need to use.
## CODE HERE ##


## Column Operations

We can do mathematic operations on and between columns using very simple syntax. Let's try multiplying the values in loan_amount_000s by 1000. Note that pandas makes it easy to apply element calculations without having to loop thorugh every row.

In [ ]:
# Multiply loan_amount_000s by 1000
raw_hmda_data['loan_amount_000s']*1000

We can easily add this calculation as a new column in the dataframe using the .assign() function. 

In [ ]:
# Add a new column loan_amount that is the output of multiplying 1000 to loan_amount_000s values
added_column_data = raw_hmda_data.assign(
    loan_amount = raw_hmda_data['loan_amount_000s']*1000
)

added_column_data

Note a few things here:
<ul>
    <li>Assign by default adds the new column to the last position.</li>
    <li>You need to save the output of the assign function to a new variable or else python just performs the assign but doesn't save the result. In this example, raw_hmda_data will not have the loan_amount column added.</li>
</ul>

In [ ]:
raw_hmda_data.columns

In [ ]:
# Try calculating the number of minorities in a population for each row by multiplying minority_population to population. Note 
# that minority_population has to be divided by 100 to get to the proper decimal format.
## CODE HERE ##


## Basic Statistics

You can also easily calculate basic statistics using pandas.

In [ ]:
# Calculate the average loan amount (321.596)
## CODE HERE ##


In [ ]:
# Calculate the sum of all loan amounts requested, including those originated, withdrawn, and denied (32,159,629)
raw_hmda_data['loan_amount_000s'].sum()

In [ ]:
# You can do this with the loc function as well to add in more conditions. Let's try calculating the average loan amount for
# applicants with income levels above 100k (447.438)
raw_hmda_data.loc[raw_hmda_data['applicant_income_000s']>100,'loan_amount_000s'].mean()

In [ ]:
# Try calculating the average loan amount for applicants with income levels below 100k (172.590)
## CODE HERE ##


In [ ]:
# Write the code to get the number of applicants as of 2016 with income levels above 100k. Use the sequence_number column for
# the official count since that gives you unique loan-level ids (37,793)
## CODE HERE ##


## Aggregating 

Another useful little function is the groupby, which splits the data into different groups depending on a variable of your choice. For example, we can group the data to show total loan amounts by Metropolitan Statistical Area/Metropolitan Division.

In [ ]:
raw_hmda_data.groupby(['msamd_name'])['loan_amount_000s'].sum()

In [ ]:
# That was pretty useful but it would be even better to order this list from largest to smallest.
## CODE HERE ##


Note that the basic structure of any groupby function is as follows:

groupby([**variable by which data is grouped**])[**variable being grouped**].**operation**

The first two parts are essentially instructions on how to group, but without instructions on how to display values. You need the operation in order to show useful data, whether that's the mean, sum, count, etc.

Note that the output from a groupby and aggregation operation varies between Pandas Series and Pandas Dataframes, which can be confusing for new users. As a rule of thumb, if you calculate more than one column of results, your result will be a Dataframe. For a single column of results, the agg function, by default, will produce a Series. For this example, we get a dataframe back since we're grouping across more than one column. This is important since there are some functions that work with dataframes but not with series.

You can force a 1-D groupby to be a dataframe if you place brackets around the variable name being grouped.

copy_training_df.groupby(['msamd_name'])[**['loan_amount_000s']**].sum().reset_index()

Why the reset_index here? If we didn't, the row index value would be `msamd_name`. In other words, if we wanted to refer to a specific row, we'd have to identify it via `msamd_name` rather than row position. By resetting the index, we're adding a leading column that numbers the rows from 0 to end. You can also achieve the same results by adding in an additional parameter "as_index=False" within the groupby parameter.

## Cleaning Data

We've worked with a raw dataset up to now. This means we have no guarantees around the quality of the data itself. There could be underlying issues that would make analysis completely useless. So what kind of issues usually affect the quality of data?
<ul>
    <li>Invalid values: Some datasets have well-known values, e.g. gender must only have “F” (Female) and “M” (Male). In this case it’s easy to detect wrong values.</li>
    <li>Formats: The most common issue. It’s possible to get values in different formats like a name written as “Name, Surname” or “Surname, Name”.</li>
    <li>Attribute dependencies: When the value of a feature depends on the value of another feature. For example, if we have some school data, the “number of students” is related to whether the person “is teacher?”. If someone is not a teacher he/she can’t have any students.</li>
    <li>Uniqueness: It’s possible to find repeated data in features that only allow unique values. For example, we can’t have two products with the same identifier.</li>
    <li>Missing values: Some features in the dataset may have blank or null values.</li>
    <li>Misspellings: Incorrectly written values.</li>
    <li>Misfielded values: When a feature contains the values of another.</li>
</ul>

Let's go through a few exercises to catch data quality issues. Remember, there are multiple ways of approaching these problems and will depend on a number of factors to get them right.

#### Mini-exercise: Identifying data issues with summary views<img style="float: left; width: 15px;" src=images/Design_Logo.png> 

In [ ]:
# Call the describe function on 'raw_hmda_df'. Show the transpose to see all columns. Remember that describe only includes
# columns with numeric values.
raw_hmda_data.describe().T

There are a lot of ways to start data cleaning. A good first step is to get some sort of summary view that lets us do broad sanity checks and get a better handle on the data. This is especially useful to avoid getting overwhelmed by extremely large datasets. Let's start with the .describe() function.

There is a lot of information here but it's up to us to figure what's useful and what is not. Take some time to sift through the table and note anything interesting that you find. Read below for some hints:

<ul>
    <li>We can immediately dismiss some of the statistical measures since they probably won't provide much useful information. In this case, many of the statistical measures like mean, standard deviation, and quartiles are difficult to interpret, especially without further context.</li>
    <li>Count is usually a good check to make sure your dataset isn't missing any values</li>
    <li>Min and Max values are extremely useful for sanity checks across dataset dimensions</li>
</ul>

Remember that data cleaning is both art and science. It's never possible to get perfect data so our job is to 80/20 the process and make sure our analysis doesn't get out of whack. Let's go ahead and tackle some of the easy data cleaning issues first. Start by dealing with missing data. When a value is missing, pandas automatically fills that in with NaN. We need to handle these since they can throw errors when we start our analysis. There are several ways to handle missing data but it's important to understand the impact of each action before you go ahead and make the transformation. For example, say we get rid of any row that has missing values. While it seems reasonable, that would actually wipe most entries since we'd be getting rid of any data that does not have all 5 dimensions of race, etc.

Instead, let's start conservatively and work our way up. 

In [ ]:
# Let's start by getting rid of duplicates. This is extremely easy to do.
unique_raw_hmda_data = raw_hmda_data.drop_duplicates()

In [ ]:
# Then, we decide to get rid of rows that have missing 'action_taken' data since that is required for our predictive analysis
# later. If we don't know the status, how can we use the data? Before we do that, let's first get an idea for how big an impact
# this makes
unique_raw_hmda_data.loc[unique_raw_hmda_data['action_taken'].isnull() == True].count().sum()

Great! It seems we have action_taken for every data point. We should also check to see if there are any missing values in 
other potentially important columns. Let's start by getting rid of any rows that are missing applicant_income_000s.

In [ ]:
# Checks for rows where applicant_income_000s is not null.
applicant_income_hmda_data = unique_raw_hmda_data.loc[raw_hmda_data['applicant_income_000s'].isnull()==False]

# Gets the number of rows 
applicant_income_hmda_data.shape[0]

In [ ]:
# Check to see what % of rows are affected
applicant_income_hmda_data.shape[0]/len(raw_hmda_data.index)

This gets rid of about 10% of the data. If we need applicant income for our analysis it may make sense to remove these rows – for now let’s assume this is what we want to do. Be aware that there are also alternative approaches to handling missing data which have varying benefits and risks (google imputation)

Try the same exercise as above but this time, check to see if we're missing any 'loan_amount_000s' data. Remember, we now have to check what's missing against the applicant_income_hmda_data since we've decided to remove any data that doesn't have applicant_income_000s values.

In [ ]:
# Create a new dataframe loan_amount_hmda_df that has all rows where loan_amount_000s is not null.
## CODE HERE ##


# Get number of rows
## CODE HERE ##


In [ ]:
# Check to see what % of rows are affected
## CODE HERE ##


You should get 1.0, which means that every row in the applicant_income_hmda_df as a loan_amount_000s value.

Next, we need to figure out how to classify whether a loan was approved or not. There may be some loans that have strange statuses that are neither approved nor denied. Based on the data dictionary, the `action_taken_name` column should give us the full range of loan application statuses.

In [ ]:
# Gets unique list of action_taken_names and how many of each there are
action_taken_list = loan_amount_hmda_data.groupby(['action_taken_name','action_taken']).size().reset_index(name='counts')

# Sorts in descending order
action_taken_list.sort_values('counts', ascending=False)

<ol>
    <li>We can immediately dismiss preapproval requests, both approved and denied, as they represent an insignificant number of applications.</li>
    <li>We could reasonably argue that we should not include "Application withdrawn by applicant" and "File closed for incompleteness" as they do not clearly indicate loan approval or denial.</li>
    <li>For loan purchased by the institution, we'd need to confirm that the loan application details pertain to the moment of origination and not the applicant's current status. In this example, let's assume the details pertain to the moment of origination so we can keep it as an approved.</li>
</ol>

Given those details, it seems that the Loan originated, Loan purchased by the institution, and Application approved but no accepted (indices 1, 2, 6) can be viewed as approvals while Application denied by financial institution (index 3) is the only denial status. To capture this information, we will append a new column called loan_application_status that contains a 0 or 1 to indicate denial or approval.
    

In [ ]:
# Add a new empty column called loan_approval_status and save to a new dataframe
loan_status_hmda_data = loan_amount_hmda_data.copy()
loan_status_hmda_data['loan_approval_status'] = ""

# If action_taken is 0, 4, or 5, set loan_approval_status to 1
## CODE HERE ##


# If action_taken is 1, set loan_approval_status to 0
loan_status_hmda_data.loc[loan_status_hmda_data['action_taken']==3,'loan_approval_status']=0

# Updates dataframe to capture loan_approval_status of 1 or 0 only
## CODE HERE ##


In [ ]:
# Let's check to make sure we get the same row counts
loan_status_hmda_data.shape[0] == action_taken_list.loc[[0,4,5,1]].counts.values.sum()

In [ ]:
# Now that we have the loan approval status column, we don't need action_taken or action_taken_name so let's drop those
loan_status_hmda_data = loan_status_hmda_data.drop(['action_taken', 'action_taken_name'],axis=1)

Overall, this part of data cleaning can get subjective as you determine which variables are important to analysis upfront. A more systematic method would be better to defend the data cleaning steps we take. Assuming applicant income is still critical so we leave out missing rows, let's take another look at the current dataset and start taking out columns we won't need for analysis.

In [ ]:
# Let's call count on the dataframe to get each column with the number of non-NA/null observations.
loan_status_hmda_data.count()

Notice that a majority of columns hover near the 82k count of rows with non-null values. However, some columns are significantly below that mark. We can reason that some missing values are not a result of bad data but rather the nature of the data itself. For example, there are 3 levels of denial_reason (denial_reason_1 - denial_reason_3). This suggests financial institutions have the option of listing multiple loan denial reasons but it's likely they only fill out one reason. In an ideal world, we would be able to take these data at face value and assume that those without multiple denial reasons truly had one. However, there really is no guarantee that all data was entered completely.

Therefore, it would seem reasonable that we can remove columns that are below a certain threshold for non-null values, assuming they wouldn't have much impact on our analysis anyways. For this exercise, let's be very conservative and get rid of columns that have less than 400 non-null values, which is roughly 0.5% of 82k.

In [ ]:
# First create a new dataframe that holds the count of non-null values per column. Reset index.
column_counts = pd.DataFrame(loan_status_hmda_data.count()).reset_index()

# Rename columns to make references clearer.
column_counts.columns = ['column','count']

# Create new variable 'columns_to_keep' that saves column names with counts greater than 22,000
columns_to_keep = column_counts.loc[column_counts['count']>400, 'column'].values

# Create a new dataframe 'min_null_hmda_data' that contains only columns_to_keep. Remember to use the loan_amount_hmda_df as the
# starting point.
## CODE HERE ##


# Check to make sure you have 60 columns left
min_null_hmda_data.shape

Perfect! Now we know that shape gives us max rows and columns but there must be some columns left that don't have 82,844 non-null values. Let's identify which those are below.

In [ ]:
# First create a new dataframe that holds the count of non-null values per column. Reset index.
column_counts = pd.DataFrame(min_null_hmda_data.count()).reset_index()

# Rename columns to make references clearer.
column_counts.columns = ['column','count']

# Create new variable 'columns_to_review' that saves column names with counts less than the the max number of rows 
## CODE HERE ##


# Create a dataframe that shows columns with less than the max number of non-null values
with_null_hmda_data = min_null_hmda_data[columns_to_review]

# Show the counts for these columns
with_null_hmda_data.count()

In [ ]:
# You should get 15 columns
with_null_hmda_data.shape[1] == 15

Before we go ahead and decide how to treat the remaining NaN (null) values, let's see if we can clear away any other columns.
<ul>
    <li>Right off the bat, we can delete columns to do with denial reasons as they are not very helpful for predicting if a loan is approved or not.</li>
    <li>A quick google search on edit status reveals that The HMDA Edits provide an explanation of specific edit questions. The data in question are either reported in error as invalid or do not agree with an expected standard (value). Edits are used to ensure data validity, accuracy and integrity, therefore should have no bearing on whether an application is approved or not.</li>
    <li>MSA/MD is the FFIEC Geocoding System that allows you to retrieve Metropolitan Statistical Area/Metropolitan Division numbers. However, MSA and MetroDiv boundaries are adjusted annually so comparisons across years will be extremely difficult to do. Therefore, we should drop any related columns for the purposes of our analysis. This would include msamd, msamd_name, hud_median_family_income, and tract_to_msamd_income</li>
    <li>Finally, while the rate_spread seems like an interesting data point we could add, there is no a sufficient volume of data to provide meaningful insights. We will drop that column for this exercise.</li>
</ul>

In [ ]:
# Creates the columns to drop
columns_to_drop = ['denial_reason_1',
       'denial_reason_2', 'denial_reason_name_1',
       'denial_reason_name_2', 'edit_status',
       'edit_status_name', 'msamd', 'msamd_name', 'hud_median_family_income','rate_spread',
       'tract_to_msamd_income']

# Drop selected columns from the min_null_hmda_data. Specify axis=1 for columns since the default is rows.
irrelevant_col_drop_hmda_data = min_null_hmda_data.drop(columns_to_drop, axis=1)

# Check that we've dropped the right number of columns. We should have 48 left
irrelevant_col_drop_hmda_data.shape

We have one more step to take before we round out our database. Since we seem to have some missing data, we'll have to replace the NaNs with some other value to ensure our analysis isn't affected. This is a significant exercise that requires a combination of client discussions, industry knowledge, and ad hoc analysis to determine. It sits a little beyond what this module can cover. Luckily, the remaining columns don't seem to have too many missing values so we can just filter for non-null rows only without cutting away too much data.

In [ ]:
# Drops any rows with null values
non_null_hmda_data = irrelevant_col_drop_hmda_data.dropna()

In [ ]:
# Check to see the impact of the drop. You should get 99.49082% of irrelevant_col_drop_hmda_data remaining.
## CODE HERE ##


Excellent! Now we have a dataframe that is clean and ready to go. Or do we??

Before we move into analysis, we need to be very aware of outliers. In statistics, an outlier is an observation point that is distant from other observations.An outlier may be due to variability in the measurement or it may indicate experimental error; the latter are sometimes excluded from the data set. An outlier can cause serious problems in statistical analyses.

Outliers would only apply to quantitative data (excluding categorical numeric values) so in our case, we need to pay attention to outliers for loan amount and applicant income. Variables like number_of_1_to_4_family_units and population would also have some outliers but are more categories that each data point belongs to, so outlier removal is not necessary.

Here, we're going to introduce a concept called Winsorization. Winsorization is a way to minimize the influence of outliers in your data by either:
<ul>
    <li>Assigning the outlier a lower weight,</li>
    <li>Changing the value so that it is close to other values in the set.</li>
</ul>

In other words, it is the process of replacing a specified number of extreme values with a smaller data value. Note that the data points are modified, not trimmed/removed. Winsorization basically works in two steps:
<ol>
    <li> First we specify the total percentage of untouched data we want to winsorize. For example, if you want to Winsorize the top 5% and bottom 5% of data points, this is equal to 100% – 5% – 5% = 90% Winsorization. A 80% Winsorization means that 10% is modified from each tail area.</li>
    <li> Then we replace the extreme values by the maximum and/or minimum values at the threshold. You could choose to add a little more to the larger/smaller values to account for their weights.</li>
</ol>

**WARNING**: Depending on how much we decide to winsorize, we could potentially introduce a lot of bias to our analysis. Therefore, finding the right range is usually an iterative process. At a grander scale, indiscriminantly winsorizing is not good practice either. We must be careful when treating for outliers since sometimes, there are hidden gems of information stored at the extremes that could lead to valuable insight. In this case, we are consioucly getting rid of extreme loan and applicant income amounts.

In [ ]:
# We will winsorize by applicant income to get rid of outliers.
# First, let's print the max and min values within the applicant income column to compare
print(non_null_hmda_data['applicant_income_000s'].max())
print(non_null_hmda_data['applicant_income_000s'].min())

In [ ]:
# Now let's winsorize the data. Note we are doing a 90% winsorization (5% from each end)
mstats.winsorize(non_null_hmda_data['applicant_income_000s'], limits=[0.05, 0.05])

Note that this returns an array that contains the winsorized data, mask, and fill_value. To access the data, we'll have to refer to it specifically.

In [ ]:
# Here we save it as a series to keep the index location
winsorized_income = pd.Series(mstats.winsorize(non_null_hmda_data['applicant_income_000s'], limits=[0.05, 0.05]).data)
winsorized_income

In [ ]:
# Then we update the values of the applicant income column in the original hmda dataframe to use the winsorized values
non_null_hmda_data.loc[:,'applicant_income_000s'] = winsorized_income.values

In [ ]:
# Now take a look at the new max and min
print(non_null_hmda_data['applicant_income_000s'].max())
print(non_null_hmda_data['applicant_income_000s'].min())

This means that we've taken the lowest 5% and highest 5% of applicant incomes and replaced them with the max and min values. This way, we keep the data while handling outliers. Try the same exercise above for loan_amount_000s below.

In [ ]:
# First, let's print the max and min values within the loan amount column to compare
## CODE HERE ##


In [ ]:
# Create a new Series 'winsorized_loan_amount' that winsorizes 90% on loan amount and saves it as a Series.
## CODE HERE ##


In [ ]:
# Then we update the values of the loan amount column in the original hmda dataframe to use the winsorized values
## CODE HERE ##


In [ ]:
# Check the new max and min. You should get a max of 675 and a min of 40
print(non_null_hmda_data['loan_amount_000s'].max())
print(non_null_hmda_data['loan_amount_000s'].min())

## Transforming data

An easy way to transform a data to see useful views is to create a pivot table. This functions very similarly to the pivot tables you make in excel.

In [ ]:
# Here we create a pivot table that groups by loan_approval_status for each category in the applicant_sex_name column and
# shows the total loan amount for each slice.
non_null_hmda_data.pivot_table(index='loan_approval_status', columns='applicant_sex_name', 
                          values='loan_amount_000s', aggfunc='sum')

In [ ]:
# Try making a pivot table that shows the average loan amount on the same conditions above
## CODE HERE ##


## Output

So we now finally have a clean dataset to work with moving forward. Usually, we want to save the cleaning process as a function so we can run it on other datasets easily. For our purposes, we want to save the clean dataset for easy access on other modules. You can even write the resulting dataframes to databases easily. In this example, we want to save the output as a csv file. This is easily done with the to_csv function.

Before we begin, navigate to the data folder within the root folder. You should only see the following files:
<img src="../images/module-3-to-csv.png" width=300px>

In [ ]:
# Call the to_csv function and save the data in the given path
non_null_hmda_data.to_csv("../data/clean_hmda_lar.csv", index=False)

Now refresh the data folder on your browser. You should see the clean file pop up!

## Extra Reading

If you want to find out more about pandas, take a look at the tutorial linked here: https://pandas.pydata.org/pandas-docs/stable/tutorials.html

# Capstone Exercises

**Write the code to solve for the following problems in the corresponding cells**
<ol>
    <li>Show the average rate of loan approval by county code.</li>
    <li>What is the average rate of loans approved for applicants that have more than 150k annual income from 2016?</li>
    <li>Show the total loan amounts approved by respondent_id in descending order.</li>
</ol>

In [ ]:
# Show the average rate of loan approval by as-of year (2012 - 2016)

In [ ]:
# What is the average rate of loans approved for applicants that have more than 150k annual income from 2016?

In [ ]:
# Show the total loan amounts approved by respondent_id in descending order.